## Preamble

### Import libraries

In [1]:
import os, sys

# Import Pandas
import pandas as pd

# Import Plotly and Cufflinks
# Plotly username and API key should be set in environment variables
import plotly
plotly.tools.set_credentials_file(username=os.environ['PLOTLY_USERNAME'], api_key=os.environ['PLOTLY_KEY'])
import plotly.graph_objs as go
import cufflinks as cf

# Import numpy
import numpy as np

## Step 1:

### Import CSV containing photovoltaic performance of solar cells into Pandas Data Frame object

In [2]:
# Import module to read in secure data
sys.path.append('../data/NREL')
import retrieve_data as rd

In [3]:
solar = rd.retrieve_dirks_sheet()

## Step 2:

### Clean the data for inconsistencies

In [4]:
sys.path.append('utils')
import process_data as prd

In [5]:
prd.clean_data(solar)

## Step 3:

### Import functions from utils and define notebook-specific functions

In [6]:
import degradation_utils as du

In [7]:
def get_mode_correlation_percent(df, mode_1, mode_2, weighted):
    """
    Return the percent of rows where two modes are seen together
    
    Args:
        df (DataFrame): Pandas DataFrame that has been cleaned using the clean_data function
        mode_1 (string): Degradation mode to find in the DataFrame in pairing with mode_2
        mode_2 (string): Degradation mode to find in the DataFrame in pairing with mode_1
        weighted (bool): If true, count all modules in a system as degrading
                         If false, count a system as one degrading module
    Returns:
        float: The percentage of modules with both specified degradation modes
    """
    # Calculate total number of modules
    total_modules = du.get_total_modules(df, weighted)
    if total_modules == 0:
        return 0
        
    if weighted:
        single_modules = len(df[(df['System/module'] == 'Module') & (df[mode_1] == 1) & (df[mode_2] == 1)])
        
        specified = df[(df['System/module'] != 'System') | (df['No.modules'].notnull())]
        systems = specified[(specified['System/module'] != 'Module') &
                            (specified[mode_1] == 1) & (specified[mode_2] == 1)]['No.modules'].sum()
        
        total = single_modules + systems
        return float(total) / total_modules
    else:
        return float(len((df[(df[mode_1] == 1) & (df[mode_2] == 1)]))) / total_modules

In [8]:
def get_heatmap_data(df, modes, weighted):
    """
    Returns a DataFrame used to construct a heatmap based on frequency of two degradation modes appearing together
    
    Args:
        df (DataFrame): A *cleaned* DataFrame containing the data entries to check modes from
        modes (List of String): A list of all modes to check for in the DataFrame
        weighted (bool): If true, count all modules in a system as degrading
                         If false, count a system as one degrading module
    Returns:
        heatmap (DataFrame): DataFrame containing all of degradation modes correlation frequency results
    """
    # Initialize DataFrame to hold heatmap data
    heatmap = pd.DataFrame(data=None, columns=modes, index=modes)

    # Calculate all single mode percentages
    mode_percentages = {}
    for mode in modes:
        mode_percentages[mode] = du.get_mode_percentage(df, mode, weighted)
    
    # Iterate through every pair of modes
    for mode_1 in modes:
        for mode_2 in modes:
            if mode_1 == mode_2:
                heatmap.set_value(mode_1, mode_2, np.nan)
            else:
                print(mode_1 + " & " + mode_2)
                
                heatmap_reflection = heatmap.at[mode_2, mode_1]
                
                # If already calculated the reflection, save and skip
                if (not pd.isnull(heatmap_reflection)):
                    heatmap.set_value(mode_1, mode_2, heatmap_reflection)
                    print('Skip, already calculated')
                    continue
                
                percentage_1 = mode_percentages[mode_1]
                percentage_2 = mode_percentages[mode_2]
                
                print('Percentage 1: ' + str(percentage_1))
                print('Percentage 2: ' + str(percentage_2))
                
                if (percentage_1 == 0 or percentage_2 == 0):
                    heatmap.set_value(mode_1, mode_2, 0)
                    continue

                percentage_both = get_mode_correlation_percent(df, mode_1, mode_2, weighted)
                print('Percentage Both: ' + str(percentage_both))

                result = float(percentage_both) / (percentage_1 * percentage_2)
                print('Result: ' + str(result))

                heatmap.set_value(mode_1, mode_2, result)
                
    return heatmap

## Step 4: Generate heatmaps of correlation frequency between degradation modes

### Calculation
Find the correlation strength of all pairs of degradation modes by using the following formula:
P(Degradation mode A & Degradation mode B) / P(Degradation mode A)P(Degradation mode B)

### Weighted: Multiply data entries for module systems by number of modules
Number of degrading modules = # of degrading single modules + (# of degrading systems · # of modules per degrading system)

Total number of modules = # of single modules + (# of systems · # of modules per system)

P(Degradation mode X) = Number of degrading modules / Total number of modules

#### Generate heatmap for the entire dataset, regardless of time

In [9]:
modes = ['Encapsulant discoloration', 'Major delamination', 'Minor delamination',
         'Backsheet other', 'Internal circuitry discoloration', 'Hot spots', 'Fractured cells',
         'Diode/J-box problem', 'Glass breakage', 'Permanent soiling', 'Frame deformation']

sys_heatmap_all = get_heatmap_data(solar, modes, True)

Encapsulant discoloration & Major delamination
Percentage 1: 0.236915056704
Percentage 2: 0.888442213156
Percentage Both: 0.230479788535
Result: 1.09499213333
Encapsulant discoloration & Minor delamination
Percentage 1: 0.236915056704
Percentage 2: 8.49843266806e-08
Percentage Both: 0.0
Result: 0.0
Encapsulant discoloration & Backsheet other
Percentage 1: 0.236915056704
Percentage 2: 0.00813317003198
Percentage Both: 0.00813317003198
Result: 4.22092210564
Encapsulant discoloration & Internal circuitry discoloration
Percentage 1: 0.236915056704
Percentage 2: 0.467244592949
Percentage Both: 0.0100421729622
Result: 0.0907174325493
Encapsulant discoloration & Hot spots
Percentage 1: 0.236915056704
Percentage 2: 0.68649463754
Percentage Both: 0.228753671875
Result: 1.40649522599
Encapsulant discoloration & Fractured cells
Percentage 1: 0.236915056704
Percentage 2: 0.00278884566435
Percentage Both: 0.00265278575733
Result: 4.01499523183
Encapsulant discoloration & Diode/J-box problem
Percent

In [10]:
sys_heatmap_all

,Encapsulant discoloration,Major delamination,Minor delamination,Backsheet other,Internal circuitry discoloration,Hot spots,Fractured cells,Diode/J-box problem,Glass breakage,Permanent soiling,Frame deformation
Encapsulant discoloration,NaN,1.09499,0,4.22092,0.0907174,1.4065,4.015,4.20874,3.98584,3.90533,4.1618
Major delamination,1.09499,NaN,0,0.978916,1.11964,1.1218,1.12553,1.1001,0.858912,1.04128,1.12166
Minor delamination,0,0,NaN,0,0,0,0,0,73.992,0,0
Backsheet other,4.22092,0.978916,0,NaN,1.86136,1.45661,0,7.63091,64.3516,0,0
Internal circuitry discoloration,0.0907174,1.11964,0,1.86136,NaN,1.44681,0.104349,0.182125,1.58939,0.000185497,0.023004
Hot spots,1.4065,1.1218,0,1.45661,1.44681,NaN,1.44602,1.42105,0.84153,0,1.45162
Fractured cells,4.015,1.12553,0,0,0.104349,1.44602,NaN,0,0.108228,0.0103594,0
Diode/J-box problem,4.20874,1.1001,0,7.63091,0.182125,1.42105,0,NaN,6.17436,0,0
Glass breakage,3.98584,0.858912,73.992,64.3516,1.58939,0.84153,0.108228,6.17436,NaN,68.4511,0.00787428
Permanent soiling,3.90533,1.04128,0,0,0.000185497,0,0.0103594,0,68.4511,NaN,0.527851


In [11]:
sys_heatmap_all.iplot(kind='heatmap',colorscale='spectral',
                                filename='sys-heatmap-all', margin=(200,150,120,30))

#### Generate heatmap for the dataset of all modules installed before 2000

In [12]:
modes = ['Encapsulant discoloration', 'Major delamination', 'Minor delamination',
         'Backsheet other', 'Internal circuitry discoloration', 'Hot spots', 'Fractured cells',
         'Diode/J-box problem', 'Glass breakage', 'Permanent soiling', 'Frame deformation']

specified = solar[solar['Begin.Year'] < 2000]

sys_heatmap_pre_2000 = get_heatmap_data(specified, modes, True)

Encapsulant discoloration & Major delamination
Percentage 1: 0.067517053776
Percentage 2: 0.946678967346
Percentage Both: 0.0636983482767
Result: 0.996579524123
Encapsulant discoloration & Minor delamination
Percentage 1: 0.067517053776
Percentage 2: 0.0
Encapsulant discoloration & Backsheet other
Percentage 1: 0.067517053776
Percentage 2: 0.00951236408476
Percentage Both: 0.00951236408476
Result: 14.811072819
Encapsulant discoloration & Internal circuitry discoloration
Percentage 1: 0.067517053776
Percentage 2: 0.626507928105
Percentage Both: 0.013503753609
Result: 0.319237904359
Encapsulant discoloration & Hot spots
Percentage 1: 0.067517053776
Percentage 2: 0.672691704867
Percentage Both: 0.0599501563733
Result: 1.31995998335
Encapsulant discoloration & Fractured cells
Percentage 1: 0.067517053776
Percentage 2: 2.73140185051e-05
Percentage Both: 2.73140185051e-05
Result: 14.811072819
Encapsulant discoloration & Diode/J-box problem
Percentage 1: 0.067517053776
Percentage 2: 0.0631983

In [13]:
sys_heatmap_pre_2000

,Encapsulant discoloration,Major delamination,Minor delamination,Backsheet other,Internal circuitry discoloration,Hot spots,Fractured cells,Diode/J-box problem,Glass breakage,Permanent soiling,Frame deformation
Encapsulant discoloration,NaN,0.99658,0,14.8111,0.319238,1.31996,14.8111,14.7075,14.8056,13.8554,0.95002
Major delamination,0.99658,NaN,0,1.0563,1.05075,1.05555,1.05632,0.998371,0.851148,0.988131,0
Minor delamination,0,0,NaN,0,0,0,0,0,0,0,0
Backsheet other,14.8111,1.0563,0,NaN,1.59613,1.48653,0,15.8228,58.8634,0,0
Internal circuitry discoloration,0.319238,1.05075,0,1.59613,NaN,1.47647,0,0.329389,1.26812,0,0.102381
Hot spots,1.31996,1.05555,0,1.48653,1.47647,NaN,0,1.39842,0.832362,0,0
Fractured cells,14.8111,1.05632,0,0,0,0,NaN,0,11.8221,0,0
Diode/J-box problem,14.7075,0.998371,0,15.8228,0.329389,1.39842,0,NaN,11.9121,0,0
Glass breakage,14.8056,0.851148,0,58.8634,1.26812,0.832362,11.8221,11.9121,NaN,55.064,3.77569
Permanent soiling,13.8554,0.988131,0,0,0,0,0,0,55.064,NaN,251.496


In [14]:
sys_heatmap_pre_2000.iplot(kind='heatmap',colorscale='spectral',
                                filename='sys-heatmap-pre-2000', margin=(200,150,120,30))

#### Generate heatmap for the dataset of all modules installed post 2000

In [15]:
modes = ['Encapsulant discoloration', 'Major delamination', 'Minor delamination',
         'Backsheet other', 'Internal circuitry discoloration', 'Hot spots', 'Fractured cells',
         'Diode/J-box problem', 'Glass breakage', 'Permanent soiling', 'Frame deformation']

specified = solar[solar['Begin.Year'] >= 2000]

sys_heatmap_post_2000 = get_heatmap_data(specified, modes, True)

Encapsulant discoloration & Major delamination
Percentage 1: 0.728248540003
Percentage 2: 0.719529658385
Percentage Both: 0.714224011912
Result: 1.36303222747
Encapsulant discoloration & Minor delamination
Percentage 1: 0.728248540003
Percentage 2: 3.31478600073e-07
Percentage Both: 0.0
Result: 0.0
Encapsulant discoloration & Backsheet other
Percentage 1: 0.728248540003
Percentage 2: 0.00413287518571
Percentage Both: 0.00413287518571
Result: 1.37315757612
Encapsulant discoloration & Internal circuitry discoloration
Percentage 1: 0.728248540003
Percentage 2: 0.00530730386577
Percentage Both: 1.98887160044e-06
Result: 0.000514580317076
Encapsulant discoloration & Hot spots
Percentage 1: 0.728248540003
Percentage 2: 0.726530486418
Percentage Both: 0.718362853713
Result: 1.35772058216
Encapsulant discoloration & Fractured cells
Percentage 1: 0.728248540003
Percentage 2: 0.0107985783546
Percentage Both: 0.0102678811159
Result: 1.30567360646
Encapsulant discoloration & Diode/J-box problem
Pe

In [16]:
sys_heatmap_post_2000

,Encapsulant discoloration,Major delamination,Minor delamination,Backsheet other,Internal circuitry discoloration,Hot spots,Fractured cells,Diode/J-box problem,Glass breakage,Permanent soiling,Frame deformation
Encapsulant discoloration,NaN,1.36303,0,1.37316,0.00051458,1.35772,1.30567,1.37316,0.208752,1.04912,1.35603
Major delamination,1.36303,NaN,0,0.000111469,1.38893,1.3754,1.38975,1.38978,0.211281,1.05982,1.38728
Minor delamination,0,0,NaN,0,0,0,0,0,290.634,0,0
Backsheet other,1.37316,0.000111469,0,NaN,0,1.37618,0,0,0,0,0
Internal circuitry discoloration,0.00051458,1.38893,0,0,NaN,1.37546,9.25412,0.000956338,0,0.273336,2.00851
Hot spots,1.35772,1.3754,0,1.37618,1.37546,NaN,1.37636,1.37639,1.14568,0,1.37391
Fractured cells,1.30567,1.38975,0,0,9.25412,1.37636,NaN,0,0,0.0447799,0
Diode/J-box problem,1.37316,1.38978,0,0,0.000956338,1.37639,0,NaN,0.000368784,0,0
Glass breakage,0.208752,0.211281,290.634,0,0,1.14568,0,0.000368784,NaN,221.63,0
Permanent soiling,1.04912,1.05982,0,0,0.273336,0,0.0447799,0,221.63,NaN,0.00107989


In [17]:
sys_heatmap_post_2000.iplot(kind='heatmap',colorscale='spectral',
                                filename='sys-heatmap-post-2000', margin=(200,150,120,30))

### Unweighted: Count module systems as single module
Number of degrading modules = # of degrading single modules + # of degrading systems

Total number of modules = # of single modules + # of systems

P(Degradation mode X) = Number of degrading modules / Total number of modules

In [18]:
modes = ['Encapsulant discoloration', 'Major delamination', 'Minor delamination',
         'Backsheet other', 'Internal circuitry discoloration', 'Hot spots', 'Fractured cells',
         'Diode/J-box problem', 'Glass breakage', 'Permanent soiling', 'Frame deformation']

modes_heatmap_all = get_heatmap_data(solar, modes, False)

Encapsulant discoloration & Major delamination
Percentage 1: 0.336320840123
Percentage 2: 0.649194278472
Percentage Both: 0.282998370451
Result: 1.29615062012
Encapsulant discoloration & Minor delamination
Percentage 1: 0.336320840123
Percentage 2: 9.05305087815e-05
Percentage Both: 0.0
Result: 0.0
Encapsulant discoloration & Backsheet other
Percentage 1: 0.336320840123
Percentage 2: 0.0307803729857
Percentage Both: 0.0307803729857
Result: 2.9733512786
Encapsulant discoloration & Internal circuitry discoloration
Percentage 1: 0.336320840123
Percentage 2: 0.265707043274
Percentage Both: 0.0342205323194
Result: 0.382939278811
Encapsulant discoloration & Hot spots
Percentage 1: 0.336320840123
Percentage 2: 0.446949121854
Percentage Both: 0.218721709216
Result: 1.45505705673
Encapsulant discoloration & Fractured cells
Percentage 1: 0.336320840123
Percentage 2: 0.0413724425131
Percentage Both: 0.0375701611443
Result: 2.70008923549
Encapsulant discoloration & Diode/J-box problem
Percentage 1

In [19]:
modes_heatmap_all

,Encapsulant discoloration,Major delamination,Minor delamination,Backsheet other,Internal circuitry discoloration,Hot spots,Fractured cells,Diode/J-box problem,Glass breakage,Permanent soiling,Frame deformation
Encapsulant discoloration,NaN,1.29615,0,2.97335,0.382939,1.45506,2.70009,2.94425,2.48781,1.9492,2.51442
Major delamination,1.29615,NaN,0,1.31385,1.47004,1.48951,1.53363,1.51273,1.2505,0.986978,1.42317
Minor delamination,0,0,NaN,0,0,0,0,0,17.1788,0,0
Backsheet other,2.97335,1.31385,0,NaN,3.21008,2.21107,0,7.65832,14.6525,0,0
Internal circuitry discoloration,0.382939,1.47004,0,3.21008,NaN,2.1078,0.329413,0.933211,1.83202,0.0418171,0.359991
Hot spots,1.45506,1.48951,0,2.21107,2.1078,NaN,1.06239,1.74283,1.14479,0,2.06715
Fractured cells,2.70009,1.53363,0,0,0.329413,1.06239,NaN,0,1.80434,0.089521,0
Diode/J-box problem,2.94425,1.51273,0,7.65832,0.933211,1.74283,0,NaN,4.11956,0,0
Glass breakage,2.48781,1.2505,17.1788,14.6525,1.83202,1.14479,1.80434,4.11956,NaN,11.0072,0.149381
Permanent soiling,1.9492,0.986978,0,0,0.0418171,0,0.089521,0,11.0072,NaN,1.27477


In [20]:
modes_heatmap_all.iplot(kind='heatmap',colorscale='spectral',
                         filename='modes-heatmap-all', margin=(200,150,120,30))

#### Generate heatmap for the dataset of all modules installed before 2000

In [21]:
modes = ['Encapsulant discoloration', 'Major delamination', 'Minor delamination',
         'Backsheet other', 'Internal circuitry discoloration', 'Hot spots', 'Fractured cells',
         'Diode/J-box problem', 'Glass breakage', 'Permanent soiling', 'Frame deformation']

specified = solar[solar['Begin.Year'] < 2000]

modes_heatmap_pre_2000 = get_heatmap_data(specified, modes, False)

Encapsulant discoloration & Major delamination
Percentage 1: 0.24806312769
Percentage 2: 0.759397417504
Percentage Both: 0.202869440459
Result: 1.07692460671
Encapsulant discoloration & Minor delamination
Percentage 1: 0.24806312769
Percentage 2: 0.0
Encapsulant discoloration & Backsheet other
Percentage 1: 0.24806312769
Percentage 2: 0.0417503586801
Percentage Both: 0.0417503586801
Result: 4.03123192597
Encapsulant discoloration & Internal circuitry discoloration
Percentage 1: 0.24806312769
Percentage 2: 0.396556671449
Percentage Both: 0.0533715925395
Result: 0.542553645608
Encapsulant discoloration & Hot spots
Percentage 1: 0.24806312769
Percentage 2: 0.421520803443
Percentage Both: 0.0890961262554
Result: 0.852074549362
Encapsulant discoloration & Fractured cells
Percentage 1: 0.24806312769
Percentage 2: 0.0342898134864
Percentage Both: 0.0342898134864
Result: 4.03123192597
Encapsulant discoloration & Diode/J-box problem
Percentage 1: 0.24806312769
Percentage 2: 0.120659971306
Perce

In [22]:
modes_heatmap_pre_2000

,Encapsulant discoloration,Major delamination,Minor delamination,Backsheet other,Internal circuitry discoloration,Hot spots,Fractured cells,Diode/J-box problem,Glass breakage,Permanent soiling,Frame deformation
Encapsulant discoloration,NaN,1.07692,0,4.03123,0.542554,0.852075,4.03123,3.9785,3.65607,3.22872,0.879542
Major delamination,1.07692,NaN,0,1.30778,1.25776,1.30249,1.31683,1.29335,1.15864,1.04859,0
Minor delamination,0,0,NaN,0,0,0,0,0,0,0,0
Backsheet other,4.03123,1.30778,0,NaN,2.51304,2.35606,0,8.23079,11.753,0,0
Internal circuitry discoloration,0.542554,1.25776,0,2.51304,NaN,2.23589,0,0.899539,1.33552,0,0.550191
Hot spots,0.852075,1.30249,0,2.35606,2.23589,NaN,0,1.63047,1.16009,0,0
Fractured cells,4.03123,1.31683,0,0,0,0,NaN,0,2.36858,0,0
Diode/J-box problem,3.9785,1.29335,0,8.23079,0.899539,1.63047,0,NaN,4.10882,0,0
Glass breakage,3.65607,1.15864,0,11.753,1.33552,1.16009,2.36858,4.10882,NaN,9.39118,2.57314
Permanent soiling,3.22872,1.04859,0,0,0,0,0,0,9.39118,NaN,24.6414


In [23]:
modes_heatmap_pre_2000.iplot(kind='heatmap',colorscale='spectral',
                                filename='modes-heatmap-pre-2000', margin=(200,150,120,30))

#### Generate heatmap for the dataset of all modules installed post 2000

In [24]:
modes = ['Encapsulant discoloration', 'Major delamination', 'Minor delamination',
         'Backsheet other', 'Internal circuitry discoloration', 'Hot spots', 'Fractured cells',
         'Diode/J-box problem', 'Glass breakage', 'Permanent soiling', 'Frame deformation']

specified = solar[solar['Begin.Year'] >= 2000]

modes_heatmap_post_2000 = get_heatmap_data(specified, modes, False)

Encapsulant discoloration & Major delamination
Percentage 1: 0.487721021611
Percentage 2: 0.461198428291
Percentage Both: 0.420432220039
Result: 1.86911790414
Encapsulant discoloration & Minor delamination
Percentage 1: 0.487721021611
Percentage 2: 0.00024557956778
Percentage Both: 0.0
Result: 0.0
Encapsulant discoloration & Backsheet other
Percentage 1: 0.487721021611
Percentage 2: 0.0120333988212
Percentage Both: 0.0120333988212
Result: 2.05035246727
Encapsulant discoloration & Internal circuitry discoloration
Percentage 1: 0.487721021611
Percentage 2: 0.0419941060904
Percentage Both: 0.00147347740668
Result: 0.0719421918341
Encapsulant discoloration & Hot spots
Percentage 1: 0.487721021611
Percentage 2: 0.490913555992
Percentage Both: 0.440815324165
Result: 1.84111189532
Encapsulant discoloration & Fractured cells
Percentage 1: 0.487721021611
Percentage 2: 0.053536345776
Percentage Both: 0.0432220039293
Result: 1.65533043229
Encapsulant discoloration & Diode/J-box problem
Percentage

In [25]:
modes_heatmap_post_2000

,Encapsulant discoloration,Major delamination,Minor delamination,Backsheet other,Internal circuitry discoloration,Hot spots,Fractured cells,Diode/J-box problem,Glass breakage,Permanent soiling,Frame deformation
Encapsulant discoloration,NaN,1.86912,0,2.05035,0.0719422,1.84111,1.65533,2.04503,0.0788597,0.151878,1.78729
Major delamination,1.86912,NaN,0,0.0442503,2.04147,2.02617,2.14837,2.12884,0.0833948,0.040153,2.08644
Minor delamination,0,0,NaN,0,0,0,0,0,78.3077,0,0
Backsheet other,2.05035,0.0442503,0,NaN,0,1.95387,0,0,0,0,0
Internal circuitry discoloration,0.0719422,2.04147,0,0,NaN,1.90598,4.36933,0.247406,0,1.32294,2.15664
Hot spots,1.84111,2.02617,0,1.95387,1.90598,NaN,2.02767,1.99469,1.56694,0,1.96015
Fractured cells,1.65533,2.14837,0,0,4.36933,2.02767,NaN,0,0,0.345906,0
Diode/J-box problem,2.04503,2.12884,0,0,0.247406,1.99469,0,NaN,0.203397,0,0
Glass breakage,0.0788597,0.0833948,78.3077,0,0,1.56694,0,0.203397,NaN,1.45014,0
Permanent soiling,0.151878,0.040153,0,0,1.32294,0,0.345906,0,1.45014,NaN,0.0569113


In [26]:
modes_heatmap_post_2000.iplot(kind='heatmap',colorscale='spectral',
                                filename='modes-heatmap-post-2000', margin=(200,150,120,30))